# Fraud Detection with Graph databases and Machine Learning

## Importing the required Python libraries

In [27]:
import numpy as np
import pandas as pd
from py2neo import Graph
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from imblearn.over_sampling import SMOTE 
from collections import Counter

In [ ]:
## Loading and exploring the banksim dataset 

In [28]:
banksim_df = pd.read_csv("../data/bs140513_032310.csv")
banksim_df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [3]:
banksim_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 10 columns):
step           594643 non-null int64
customer       594643 non-null object
age            594643 non-null object
gender         594643 non-null object
zipcodeOri     594643 non-null object
merchant       594643 non-null object
zipMerchant    594643 non-null object
category       594643 non-null object
amount         594643 non-null float64
fraud          594643 non-null int64
dtypes: float64(1), int64(2), object(7)
memory usage: 45.4+ MB


Viewing the split of the output classes - fraudulent and genuine transactions

In [4]:
banksim_df['fraud'].value_counts()

0    587443
1      7200
Name: fraud, dtype: int64

In [5]:
# Number of unique values per column in the banksim dataset
banksim_df.nunique()

step             180
customer        4112
age                8
gender             4
zipcodeOri         1
merchant          50
zipMerchant        1
category          15
amount         23767
fraud              2
dtype: int64

## Preprocessing the data 

In [6]:
# Obtaining the number of null values in each column
banksim_df.isna().sum()

step           0
customer       0
age            0
gender         0
zipcodeOri     0
merchant       0
zipMerchant    0
category       0
amount         0
fraud          0
dtype: int64

Looks like there are no null values in the dataset.

In [12]:
# Retrieving the class attribute from the dataframe
Y_before_smote = banksim_df['fraud']
Y_before_smote.head()

0    0
1    0
2    0
3    0
4    0
Name: fraud, dtype: int64

In [69]:
'''
Removing unwanted columns
Since zipcodeOri and zipMerchant have the same value for all the rows, these columns are redundant
'''

feature_df = banksim_df.drop(['step', 'customer', 'zipcodeOri', 'zipMerchant', 'fraud'], axis=1)

# Encoding the categorical variables because SMOTE cannot work with string values
label_encoder = LabelEncoder()
feature_df['gender'] = label_encoder.fit_transform(feature_df['gender'])
feature_df['age'] = label_encoder.fit_transform(feature_df['age'])
feature_df['category'] = label_encoder.fit_transform(feature_df['category'])
feature_df['merchant'] = label_encoder.fit_transform(feature_df['merchant'])


In [70]:
feature_df.head()

,age,gender,merchant,category,amount
0,4,2,30,12,4.55
1,2,2,30,12,39.68
2,4,1,18,12,26.89
3,3,2,30,12,17.25
4,5,2,30,12,35.72


In [71]:
# Handling the imbalance in the dataset using SMOTE

print('Original dataset shape %s' % Counter(Y_before_smote))

X_before_smote = feature_df
sm = SMOTE(random_state=1)

# Applying smote to dataset, result is nparray
X_after_smote, Y_after_smote = sm.fit_resample(X_before_smote, Y_before_smote)
                                               
# Converting back tp data frame from nparray
X_after_smote_df = pd.DataFrame(X_after_smote, columns=X_before_smote.columns)
Y_after_smote_df = pd.DataFrame(Y_after_smote, columns = ["fraud"])
                                               
print('dataset shape after smote %s' % Counter(Y_after_smote))

Original dataset shape Counter({0: 587443, 1: 7200})
dataset shape after smote Counter({0: 587443, 1: 587443})


In [72]:
# Verifying the number of unique values in each column after SMOTE
X_after_smote_df.nunique()

age              8
gender           4
merchant        50
category        15
amount      602524
dtype: int64

In [73]:
# Converting the encoded merchant values back to the original string values
X_after_smote_df['merchant'] = label_encoder.inverse_transform(X_after_smote_df['merchant'])

# One hot encoding the categorical variables
feature_df = pd.get_dummies(X_after_smote_df, columns=['age', 'gender', 'category', 'merchant'])
feature_df.head()

,amount,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_7,gender_0,...,merchant_'M732195782',merchant_'M78078399',merchant_'M840466850',merchant_'M855959430',merchant_'M857378720',merchant_'M85975013',merchant_'M923029380',merchant_'M933210764',merchant_'M97925176',merchant_'M980657600'
0,4.55,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,39.68,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,26.89,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17.25,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,35.72,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
# Standardizing the features
standard_scaler = StandardScaler()
scaled_df = pd.DataFrame(standard_scaler.fit_transform(feature_df), columns = feature_df.columns)

scaled_df.head()

,amount,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_7,gender_0,...,merchant_'M732195782',merchant_'M78078399',merchant_'M840466850',merchant_'M855959430',merchant_'M857378720',merchant_'M85975013',merchant_'M923029380',merchant_'M933210764',merchant_'M97925176',merchant_'M980657600'
0,-0.431072,-0.07114,-0.35114,-0.747483,-0.592631,2.227360,-0.283103,-0.156367,-0.031681,-0.03967,...,-0.180295,-0.086587,-0.09681,-0.167794,-0.06582,-0.161329,-0.053474,-0.043598,-0.047483,-0.335042
1,-0.376549,-0.07114,-0.35114,1.337822,-0.592631,-0.448962,-0.283103,-0.156367,-0.031681,-0.03967,...,-0.180295,-0.086587,-0.09681,-0.167794,-0.06582,-0.161329,-0.053474,-0.043598,-0.047483,-0.335042
2,-0.396399,-0.07114,-0.35114,-0.747483,-0.592631,2.227360,-0.283103,-0.156367,-0.031681,-0.03967,...,-0.180295,-0.086587,-0.09681,-0.167794,-0.06582,-0.161329,-0.053474,-0.043598,-0.047483,-0.335042
3,-0.411361,-0.07114,-0.35114,-0.747483,1.687392,-0.448962,-0.283103,-0.156367,-0.031681,-0.03967,...,-0.180295,-0.086587,-0.09681,-0.167794,-0.06582,-0.161329,-0.053474,-0.043598,-0.047483,-0.335042
4,-0.382695,-0.07114,-0.35114,-0.747483,-0.592631,-0.448962,3.532282,-0.156367,-0.031681,-0.03967,...,-0.180295,-0.086587,-0.09681,-0.167794,-0.06582,-0.161329,-0.053474,-0.043598,-0.047483,-0.335042


## Training supervised learning models using intrinsic features from the dataset

In [ ]:
k_fold = KFold(n_splits=5, random_state=None, shuffle=False)


random_forest = RandomForestClassifier(max_depth=4, n_estimators=150)
svm = SVC(gamma="auto")
logistic_regression = LogisticRegression(solver='lbfgs')
scaled_df = scaled_df.values
labels = labels.values

In [13]:
# Random Forest Classifier

for train_index, test_index in k_fold.split(scaled_df):
    X_train, X_test = scaled_df[train_index], scaled_df[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    clf = random_forest.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions) * 100

    metrics = precision_recall_fscore_support(y_test, predictions, average='macro')
    precision, recall = metrics[0], metrics[1]
    f1_score = 2 * (precision * recall) / (precision + recall)

    
    print("Accuracy = {0:.4f}%".format(accuracy))
    print("Precision = {0:.4f}".format(precision))
    print("Recall = {0:.4f}".format(recall))
    print("F1 Score = {0:.4f}".format(f1_score))
    print("---------------------------------------------")

Accuracy = 98.9035%
Precision = 0.9695
Recall = 0.6267
F1 Score = 0.7612
---------------------------------------------
Accuracy = 99.0103%
Precision = 0.9802
Recall = 0.6094
F1 Score = 0.7516
---------------------------------------------
Accuracy = 99.0902%
Precision = 0.9860
Recall = 0.6118
F1 Score = 0.7551
---------------------------------------------
Accuracy = 99.1280%
Precision = 0.9825
Recall = 0.6114
F1 Score = 0.7537
---------------------------------------------
Accuracy = 99.1423%
Precision = 0.9938
Recall = 0.6019
F1 Score = 0.7498
---------------------------------------------


In [13]:
# SVM Classifier
'''
for train_index, test_index in k_fold.split(scaled_df):
    X_train, X_test = scaled_df[train_index], scaled_df[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    clf = svm.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions) * 100

    metrics = precision_recall_fscore_support(y_test, predictions, average='macro')
    precision, recall = metrics[0], metrics[1]
    f1_score = 2 * (precision * recall) / (precision + recall)

    
    print("Accuracy = {0:.4f}%".format(accuracy))
    print("Precision = {0:.4f}".format(precision))
    print("Recall = {0:.4f}".format(recall))
    print("F1 Score = {0:.4f}".format(f1_score))
    print("---------------------------------------------")'''

Finished training
Accuracy = 99.3610%
Precision = 0.9014
Recall = 0.8650
F1 Score = 0.8828
---------------------------------------------
Finished training
Accuracy = 99.5434%
Precision = 0.9360
Recall = 0.8705
F1 Score = 0.9021
---------------------------------------------
Finished training
Accuracy = 99.5628%
Precision = 0.9406
Recall = 0.8588
F1 Score = 0.8978
---------------------------------------------
Finished training
Accuracy = 99.6149%
Precision = 0.9558
Recall = 0.8603
F1 Score = 0.9055
---------------------------------------------
Finished training
Accuracy = 99.6241%
Precision = 0.9734
Recall = 0.8432
F1 Score = 0.9036
---------------------------------------------


In [15]:
# Logistic Regression Classifier
for train_index, test_index in k_fold.split(scaled_df):
    X_train, X_test = scaled_df[train_index], scaled_df[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    clf = logistic_regression.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions) * 100

    metrics = precision_recall_fscore_support(y_test, predictions, average='macro')
    precision, recall = metrics[0], metrics[1]
    f1_score = 2 * (precision * recall) / (precision + recall)

    
    print("Accuracy = {0:.4f}%".format(accuracy))
    print("Precision = {0:.4f}".format(precision))
    print("Recall = {0:.4f}".format(recall))
    print("F1 Score = {0:.4f}".format(f1_score))
    print("---------------------------------------------")

/Users/Ares/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Accuracy = 99.3753%
Precision = 0.9029
Recall = 0.8694
F1 Score = 0.8858
---------------------------------------------


/Users/Ares/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Accuracy = 99.5367%
Precision = 0.9299
Recall = 0.8744
F1 Score = 0.9013
---------------------------------------------


/Users/Ares/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Accuracy = 99.5720%
Precision = 0.9406
Recall = 0.8634
F1 Score = 0.9003
---------------------------------------------


/Users/Ares/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Accuracy = 99.6216%
Precision = 0.9597
Recall = 0.8599
F1 Score = 0.9071
---------------------------------------------
Accuracy = 99.6149%
Precision = 0.9669
Recall = 0.8439
F1 Score = 0.9012
---------------------------------------------


/Users/Ares/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


## Extracting network features

In [23]:
# Connecting to the Neo4j database
graph = Graph(password="root")

# Query to fetch the network features from Neo4j
query = """
MATCH (p:Placeholder)
RETURN p.id AS id, p.degree AS degree, p.pagerank as pagerank, p.community AS community 
"""

data = graph.run(query)

records = {}

for record in data:
    records[record['id']] = {'degree': record['degree'], 'pagerank': record['pagerank'], 'community': record['community']}


In [24]:
banksim_df = pd.read_csv("../data/bs140513_032310.csv")


In [25]:
def load_degree(record):
    return records[record.split("'")[1]]['degree']
def load_community(record):
    return str(records[record.split("'")[1]]['community'])
def load_pagerank(record):
    return records[record.split("'")[1]]['pagerank']

In [26]:
banksim_df['merchant_degree'] = banksim_df['merchant'].apply(load_degree)
banksim_df['customer_degree'] = banksim_df['customer'].apply(load_degree)
banksim_df['merchant_pagerank'] = banksim_df['merchant'].apply(load_pagerank)
banksim_df['customer_pagerank'] = banksim_df['customer'].apply(load_pagerank)
banksim_df['merchant_community'] = banksim_df['merchant'].apply(load_community)
banksim_df['customer_community'] = banksim_df['customer'].apply(load_community)

In [27]:
banksim_df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud,merchant_degree,customer_degree,customer_pagerank,merchant_pagerank,merchant_community,customer_community
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0,3929,7,0.15,46.541354,598811,598811
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0,3929,12,0.15,46.541354,598811,598846
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0,3573,14,0.15,41.391096,598846,598811
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0,3929,11,0.15,46.541354,598811,598846
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0,3929,14,0.15,46.541354,598811,598811
